In [ ]:
pip install nltk


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import nltk

# Descargar los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



In [ ]:
# Cargar el archivo CSV
file_path = './wiki_movie_plots_deduped.csv'
df = pd.read_csv(file_path)

# Inicializar el lematizador y stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Función para normalizar el texto
def normalize_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación y caracteres especiales
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenizar
    tokens = word_tokenize(text)
    # Eliminar stopwords y lematizar las palabras
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

# Aplicar la normalización a las tramas de las películas
df['Normalized_Plot'] = df['Plot'].apply(normalize_text)

# Crear el índice invertido
inverted_index = defaultdict(set)

for idx, row in df.iterrows():
    title = row['Title']
    for word in row['Normalized_Plot']:
        inverted_index[word].add(title)



In [ ]:
# Consultar el índice invertido para la palabra 'cyborg'
word = 'man'  # Palabra que quieres buscar
movies_with_word = inverted_index[word] if word in inverted_index else set()

# Mostrar solo las primeras 5 películas
print(f"Primeras 5 películas que contienen la palabra '{word}':")
print(list(movies_with_word)[:5])


###  PARTE 2: Usar Whoosh para Indexacion 

In [ ]:
pip install whoosh


In [28]:
import pandas as pd
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
import os
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Cargar el archivo CSV
file_path = './wiki_movie_plots_deduped.csv'
df = pd.read_csv(file_path)

# Inicializar el lematizador y stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Función para normalizar el texto
def normalize_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación y caracteres especiales
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenizar
    tokens = word_tokenize(text)
    # Eliminar stopwords y lematizar las palabras
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

# Aplicar la normalización a las tramas de las películas
df['Normalized_Plot'] = df['Plot'].apply(normalize_text)

# Definir el esquema del índice con los campos 'Title' y 'Plot'
schema = Schema(
    title=TEXT(stored=True),  # Título de la película
    plot=TEXT(stored=True)    # Trama de la película
)

# Directorio donde se guardará el índice
index_dir = 'movie_index'

# Verificar si ya existe un índice en el directorio. Si no, crearlo
if not os.path.exists(index_dir):
    os.mkdir(index_dir)
    ix = create_in(index_dir, schema)
else:
    ix = open_dir(index_dir)

# Función para agregar películas al índice
def add_movies_to_index(movies):
    writer = ix.writer()  # Crear el escritor para agregar documentos
    
    for idx, row in movies.iterrows():
        title = row['Title']
        plot = row['Normalized_Plot']  # Usamos la trama normalizada
        
        # Convertir la lista de palabras de la trama de nuevo a una cadena
        plot_text = " ".join(plot)
        
        # Agregar cada película como documento en el índice
        writer.add_document(title=title, plot=plot_text)
    
    # Guardar los documentos en el índice
    writer.commit()

# Agregar todas las películas al índice
add_movies_to_index(df)

# Realizar una consulta en el índice
query_string = "man"  # Palabra clave para la consulta

with ix.searcher() as searcher:
    query = QueryParser("plot", ix.schema).parse(query_string)
    
    # Ejecutar la búsqueda
    results = searcher.search(query, limit=5)  # Limitamos a 5 resultados
    
    # Mostrar los resultados
    print(f"Películas relacionadas con '{query_string}':")
    for result in results:
        print(f"Title: {result['title']}")
        print(f"Plot: {result['plot'][:200]}...")  # Mostrar solo una parte de la trama
        print("-" * 40)


Películas relacionadas con 'man':
Title: My Wrongs #8245–8249 & 117
Plot: protagonist paddy considine man tasked looking friend imogens house specifically instructed take dog rothko walk let leash putting imogens key mail slot door dont get lost man ensures wont lose rothko...
----------------------------------------
Title: The Road
Plot: man young son struggle survive global cataclysm caused extinction event scavenge supply avoid roaming gang travel road coast hope warmer year earlier man wife give birth son shortly catastrophe gradua...
----------------------------------------
Title: The Lucky Dog
Plot: hapless hero laurel thrown onto street paying rent befriended stray dog dog young man literally bump robber hardy holding someone bandit process accidentally placed victim money young man back pocket ...
----------------------------------------
Title: The Legend is Born – Ip Man
Plot: child ip man learns wing chun chan wahshun together ip tinchi ip man adopted brother lee meiwai chans

In [30]:
# Realizar una consulta en el índice
query_string = "cyborg"  # Palabra clave para la consulta

with ix.searcher() as searcher:
    query = QueryParser("plot", ix.schema).parse(query_string)
    
    # Ejecutar la búsqueda
    results = searcher.search(query, limit=5)  # Limitamos a 5 resultados
    
    # Mostrar los resultados
    print(f"Películas relacionadas con '{query_string}':")
    for result in results:
        print(f"Title: {result['title']}")
        print(f"Plot: {result['plot'][:200]}...")  # Mostrar solo una parte de la trama
        print("-" * 40)


Películas relacionadas con 'cyborg':
Title: Cyborg 3: The Recycler
Plot: set desolate post apocalyptic world thriving golden age man cyborg ended cyborg hunted part cash haje female cyborg learns doc edford margaret avery somehow pregnant search fabled city cytown find eva...
----------------------------------------
Title: Cyborg 3: The Recycler
Plot: set desolate post apocalyptic world thriving golden age man cyborg ended cyborg hunted part cash haje female cyborg learns doc edford margaret avery somehow pregnant search fabled city cytown find eva...
----------------------------------------
Title: Cyborg 3: The Recycler
Plot: set desolate post apocalyptic world thriving golden age man cyborg ended cyborg hunted part cash haje female cyborg learns doc edford margaret avery somehow pregnant search fabled city cytown find eva...
----------------------------------------
Title: Cyborg Cop II
Plot: crime bos jesse starkraven lead gang attack drug den cooperating starkravens demand assault q